In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
import re
from src.anexo_c.process_dashboard import process
from tqdm.notebook import tqdm

In [ ]:
load_dotenv()
processamentos = process()

input_dir = os.getenv('input_dir')
output_dir = os.getenv('output_dir')
invalid_dir = os.getenv('invalid_dir')

dashboard_csv = os.path.join(input_dir, "Anexo C", "Diligencia_Contratos.csv")
template_path = os.path.join(input_dir, "Anexo C", "template.xlsx")

value_cols = ['ValorDebito', 'ValorCredito', 'Movimentacao']
detalhes_cols = ['Conta_Nome','Cosif_Nome']

configs = [
    (['Total das Receitas', ' Exclusão'], '(01) Total das Receitas'),
    ([' Exclusão'], '(02) Exclusão'),
    ([' Dedução'], '(03) Dedução'),
    (['Sem efeito'],'Sem efeito no Pis Cofins'),
]

In [ ]:
dashboard = pd.read_csv(dashboard_csv, sep=';', encoding='latin1')

In [ ]:
contratos = dashboard['NumContrato'].unique().tolist()

In [ ]:
pasta = os.path.join(output_dir, "new outputs")
processamentos.extrair_numeros_de_arquivos(pasta)

In [ ]:


nao_encontrados_3 = set()
if os.path.isfile("numeros_extraidos.txt"):
    with open("numeros_extraidos.txt", "r") as f:
        nao_encontrados_3 = set(line.strip() for line in f)

# Filtre os contratos removendo aqueles que estão na lista de não encontrados
contratos = [contrato for contrato in contratos if str(contrato) not in nao_encontrados_3]

for contrato in tqdm(contratos, unit="contrato"):
    tqdm.desc = f"processando contrato: {contrato}"
    dashboard_filtro = dashboard[(dashboard['NumContrato'] == contrato)]


    dashboard_filtro = processamentos.replicar_anos_com_detalhes(dashboard_filtro,
                                                                pis_col='PIS',
                                                                ano_mes_col='AnoMes',
                                                                detalhes_cols=detalhes_cols,
                                                                value_cols=value_cols,
                                                                fill_value=0)
    final = pd.DataFrame()

# para cada filtro, gera o bloco e concatena
    for filtro, desc in configs:
        bloco = processamentos.agrupa_receitas(dashboard_filtro, filtro, desc)
        final = pd.concat([final, bloco], ignore_index=True)



    df_pis_cofins = processamentos.calcula_pis_cofins(final)



    final = pd.concat([final, df_pis_cofins], ignore_index=True)


    final = final[['Ano', 'Descrição', 'Conta_Nome', 'Cosif_Nome', 'ValorCredito', 'Movimentacao']]

    df = final.copy()
    output_path   = os.path.join(output_dir, "new outputs", f"C{contrato}.xlsx")

    processamentos.atualizar_template_pivot(
        template_path=template_path,
        output_path=output_path,
        df=df,
        contrato=contrato
    )
